In [26]:
# Download resnet.py and overwrite if it exists
!wget -O resnet.py https://raw.githubusercontent.com/tomgoldstein/loss-landscape/master/cifar10/models/resnet.py

# Import the resnet module
import resnet

center_model = resnet.ResNet18()

--2024-06-03 15:19:03--  https://raw.githubusercontent.com/tomgoldstein/loss-landscape/master/cifar10/models/resnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9883 (9.7K) [text/plain]
Saving to: ‘resnet.py’

resnet.py           100%[===================>]   9.65K  --.-KB/s    in 0s      

2024-06-03 15:19:03 (113 MB/s) - ‘resnet.py’ saved [9883/9883]



In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use datatransform from paper
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
# no data agumentation
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

dataset = CIFAR10(root='./data', train=False, download=True, transform=transform) # 163 MB

BATCH_SIZE = 100 # Big but not too big, because I think that the batch is duplicated for each model in the vmap-enselmble
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers = 2)

# criterion functional
criterion = F.cross_entropy

# parameter subspaces to investigate.
# for each string, the union of all modules that contain the string in their name will be considered.
# we will sample a random direction in that subspace + the gradient ascent and descent directions projected onto that subspace.
SUBSPACES = []

#check if we are on GPU, otherwise just proof of concept
if torch.cuda.is_available():
    NUM_DIRS = 6
    MAX_OOM = 1
    MIN_OOM = -8
else:
    NUM_DIRS = 0 # number of random directions to sample, in addition to the gradient ascent + descent, and radially in + out.
    MAX_OOM = 0 # maximum order of magnitude to sample
    MIN_OOM = -1 # minimum order of magnitude to sample

Files already downloaded and verified


Define utility functions for parameter space arithmetic. iadd is inplace, all others create a new model.

In [28]:
from copy import deepcopy

@torch.no_grad
def iadd(a:nn.Module, b:nn.Module):
    """add the parameters of b to a, inplace"""
    for a_param, b_param in zip(a.parameters(), b.parameters()):
        a_param.data.add_(b_param.data)
    return a

@torch.no_grad
def add(a_old:nn.Module, b:nn.Module):
    """add the parameters of b to a"""
    a = deepcopy(a_old)
    return iadd(a, b)

@torch.no_grad
def scale(a_old:nn.Module, b:float):
    """scale the parameters of a by b"""
    a = deepcopy(a_old)
    for a_param in a.parameters():
        a_param.data.mul_(b)
    return a

@torch.no_grad
def sub(a:nn.Module, b:nn.Module):
    """subtract the parameters of b from a"""
    neg_b = scale(b, -1)
    return iadd(neg_b, a)

@torch.no_grad
def abs(a:nn.Module):
    """return the norm of the parameters of a"""
    return torch.norm(torch.cat([param.data.flatten() for param in a.parameters()]))

@torch.no_grad
def norm(a:nn.Module):
    """normalize the parameters of a"""
    return scale(a, 1/abs(a))

@torch.no_grad
def rand_like(a: nn.Module):
    """normalised random direction in the parameter space of the model."""
    rand = deepcopy(a)
    for param in rand.parameters():
        param.data = torch.randn_like(param.data)
    return norm(rand)

@torch.no_grad
def project_to_module(a: nn.Module, target_subspace: str):
    """
    normalised projection onto the subspace of the model specified by `target_subspace`.
    subspace: Any parameter whose name contains the `target_subspace` string.
    """
    projection = deepcopy(a)
    for name, param in projection.named_parameters():
        if target_subspace not in name:
            param.data.zero_()
    return norm(projection)

Define profiling context manager: measure execution time and GPU RAM usage before and after.

In [29]:
from time import perf_counter
from contextlib import contextmanager
import torch

@contextmanager
def profiler(description, length=80, pad_char=':'):
    print('\n'+description.center(length, pad_char))
    if torch.cuda.is_available():
        print(f'GPU RAM before execution: Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    start = perf_counter()
    yield
    if torch.cuda.is_available():
        print(f'GPU RAM after execution:  Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    print(f'Finished {description} in {perf_counter() - start:.2f} s'.center(length, pad_char))

with profiler('Profiler example'):
    print('Computation goes here...')


::::::::::::::::::::::::::::::::Profiler example::::::::::::::::::::::::::::::::
Computation goes here...
::::::::::::::::::::::Finished Profiler example in 0.00 s:::::::::::::::::::::::


In [30]:
# create a model to find the gradient
grad_model = deepcopy(center_model)
grad_model = grad_model.to(device)

optimizer = torch.optim.SGD(grad_model.parameters(), lr=1)
optimizer.zero_grad()
torch.set_grad_enabled(True)

# take a single step of gradient descent
# gradient accumulate over the whole dataset
with profiler('1 step of gradient descent over the whole dataset'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        output = grad_model(data)
        loss = criterion(output, target)
        loss.backward()

optimizer.step()
# from this point on we wont need any gradients
torch.set_grad_enabled(False)


# calculate the direction of gradient decent from the center model
grad_model = grad_model.to('cpu')
dir_ascent = sub(center_model, grad_model)
dir_ascent = norm(dir_ascent)
dir_descent = scale(dir_ascent, -1)

del grad_model



:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::
:::::Finished 1 step of gradient descent over the whole dataset in 24.96 s::::::


In [31]:
radial = norm(center_model)

dirs = {
    'Ascent': dir_ascent,
    'Descent': dir_descent,
    'Radially Out': radial,
    'Radially In': scale(radial,-1),
    }

# add random directions
dirs.update({f'Random {i}': rand_like(center_model) for i in range(NUM_DIRS)})

# add projections of each dir in dirs onto each SUBSPACE
# Add projections of each direction in dirs onto each subspace
if not SUBSPACES == []:
    for dir_name, direction in list(dirs.items()):
        for subspace in SUBSPACES:
            projected_dir = project_to_module(direction, subspace)
            new_key = f'{dir_name} ⋅ P({subspace})'
            dirs[new_key] = projected_dir

# check that they are all normalised
assert all(torch.isclose(abs(d), torch.tensor(1.0), rtol=0.001) for d in dirs.values())

In [32]:
import pandas as pd
import numpy as np

# convert to base 2
POW_MIN_DIST = int(np.floor(MIN_OOM*np.log2(10)))
POW_MAX_DIST = int(np.ceil(MAX_OOM*np.log2(10)))

# will sample these distances
scales = 2. ** np.arange(POW_MIN_DIST, POW_MAX_DIST + 1)

# offset each direction by a different amount, so that we get a nice spread of distances, to avoid artefacts/robustness
offset_per_direction = 2. ** np.linspace(0, 1, len(dirs), endpoint=False)

# Initialize an empty list to hold tuples for MultiIndex
index_list = []

# Calculate distances using broadcasting
distances = np.outer(scales, offset_per_direction).T.flatten()

# Repeat directions accordingly
directions = np.repeat(list(dirs.keys()), len(scales))

# Create DataFrame directly from numpy arrays
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([directions, distances], names=['Direction', 'Distance']))

df


Empty DataFrame
Columns: []
Index: [(Ascent, 0.0625), (Ascent, 0.125), (Ascent, 0.25), (Ascent, 0.5), (Ascent, 1.0), (Descent, 0.07432544468767006), (Descent, 0.14865088937534013), (Descent, 0.29730177875068026), (Descent, 0.5946035575013605), (Descent, 1.189207115002721), (Radially Out, 0.08838834764831845), (Radially Out, 0.1767766952966369), (Radially Out, 0.3535533905932738), (Radially Out, 0.7071067811865476), (Radially Out, 1.4142135623730951), (Radially In, 0.10511205190671431), (Radially In, 0.21022410381342863), (Radially In, 0.42044820762685725), (Radially In, 0.8408964152537145), (Radially In, 1.681792830507429)]

In [33]:
# fill the dataframe with the models
for dir_name, dist in df.index:
    shift = scale(dirs[dir_name], dist)     # shift = direction * distance
    location = iadd(shift, center_model)    # location = center + shift
    df.loc[(dir_name, dist), 'Model'] = location

df

Model
Direction    Distance                                                   
Ascent       0.062500  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.125000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.250000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.500000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             1.000000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
Descent      0.074325  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.148651  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.297302  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.594604  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             1.189207  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
Radially Out 0.088388  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.176777  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.353553  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.707107  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             1.414214  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
Radially In  0.105112  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.210224  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.420448  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             0.840896  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...
             1.681793  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...

Parallel evaluation with vmap()


In [25]:
from torch.func import stack_module_state, functional_call
from torch import vmap

# add the center model
ensemble_list = [center_model] + list(df['Model'])

print(f'Ensemble size: {len(ensemble_list)}')

# stack to prepare for vmap
params, buffers = stack_module_state(ensemble_list)

with profiler(f'Moving stacked ensemble to {device}'):
    params = {name: tensor.to(device) for name, tensor in params.items()}
    buffers = {name: tensor.to(device) for name, tensor in buffers.items()}

stacked_ensemble = (params, buffers)

# Construct a "stateless" version of one of the models. It is "stateless" in
# the sense that the parameters are meta Tensors and do not have storage.
meta_model = deepcopy(center_model).to('meta')

def meta_model_loss(params_and_buffers, data, target):
    """Compute the loss of a set of params on a batch of data, via the meta model"""
    predictions = functional_call(meta_model, params_and_buffers, (data,))
    predictions = predictions.double() # double precision for better resoltion of small scales
    loss = criterion(predictions, target)
    return loss

# define a loss function that takes the stacked ensemble params, data, and target
# in_dims=(0, None, None) adds an ensemble dimension to the first argument 'params_and_buffers' but not to the other two arguments
vmap_loss = vmap(meta_model_loss, in_dims=(0, None, None))

batch_losses = []
with profiler(f'Evaluating stacked ensemble on {device}'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        batch_loss = vmap_loss(stacked_ensemble, data, target)
        batch_losses.append(batch_loss)

with profiler('Freeing data and target'):
    del data, target
    torch.cuda.empty_cache()

with profiler('Freeing stacked ensemble'):
    del stacked_ensemble
    torch.cuda.empty_cache()

loss_tensor = torch.mean(torch.stack(batch_losses), dim=0)

Ensemble size: 21

:::::::::::::::::::::::::Moving stacked ensemble to cpu:::::::::::::::::::::::::
:::::::::::::::Finished Moving stacked ensemble to cpu in 0.00 s::::::::::::::::

:::::::::::::::::::::::Evaluating stacked ensemble on cpu:::::::::::::::::::::::
::::::::::::Finished Evaluating stacked ensemble on cpu in 619.76 s:::::::::::::

::::::::::::::::::::::::::::Freeing data and target:::::::::::::::::::::::::::::
:::::::::::::::::::Finished Freeing data and target in 0.00 s:::::::::::::::::::

::::::::::::::::::::::::::::Freeing stacked ensemble::::::::::::::::::::::::::::
::::::::::::::::::Finished Freeing stacked ensemble in 0.00 s:::::::::::::::::::


In [34]:
# Convert the loss tensor to a list and unpack
center_loss, *dir_losses = loss_tensor.tolist()

# Ensure the length of dir_losses matches the DataFrame length
assert len(dir_losses) == len(df)

# Add the directional losses to the DataFrame
df['Loss'] = dir_losses

# Add the center_loss to each Direction, for Distance=0
for direction in df.index.get_level_values('Direction').unique():
    df.loc[(direction, 0.0), 'Loss'] = center_loss

df.sort_index(inplace=True)

df

Model  \
Direction    Distance                                                      
Ascent       0.000000                                                NaN   
             0.062500  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.125000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.250000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.500000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.000000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Descent      0.000000                                                NaN   
             0.074325  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.148651  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.297302  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.594604  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.189207  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Radially In  0.000000                                                NaN   
             0.105112  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.210224  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.420448  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.840896  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.681793  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Radially Out 0.000000                                                NaN   
             0.088388  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.176777  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.353553  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.707107  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.414214  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   

                            Loss  
Direction    Distance             
Ascent       0.000000   2.413582  
             0.062500   2.659799  
             0.125000   2.987322  
             0.250000   3.835409  
             0.500000   5.992914  
             1.000000  10.814508  
Descent      0.000000   2.413582  
             0.074325   2.250852  
             0.148651   2.242186  
             0.297302   2.588072  
             0.594604   3.985614  
             1.189207   7.398079  
Radially In  0.000000   2.413582  
             0.105112   2.412963  
             0.210224   2.412346  
             0.420448   2.411118  
             0.840896   2.408692  
             1.681793   2.403950  
Radially Out 0.000000   2.413582  
             0.088388   2.414105  
             0.176777   2.414629  
             0.353553   2.415683  
             0.707107   2.417811  
             1.414214   2.422147

In [35]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Plotting directly from grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Loss'],
        mode='lines+markers',
        name=direction
    ))

# Set axis labels and title with adjusted figure dimensions
fig.update_layout(
    title=f'Loss Landscape of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis_title='Distance from Center in Parameter Space',
    yaxis_title='Loss',
    legend_title='Direction',
    template='seaborn',
    height=600,
)


# Show the figure
fig.show()


Now let's measure the roughness.

In [36]:
# TODO: Add nice explanation of the roughness calculation
df['Roughness'] = np.nan

# Calculate the roughness, one direction at a time
for direction, group in df.groupby(level='Direction'):
    distA = group.index.get_level_values('Distance').to_numpy()
    losses = group['Loss'].to_numpy()

    # Named slices, so that
    # distA[x] really corresponds to the distance between A and X ∈ {A, B, C},
    # losses[x] really corresponds to the loss at X ∈ {A, B, C}
    a = 0
    b = slice(1, -1)
    c = slice(2, None)

    # Ensure distances and slices are correct
    assert np.all(distA[c] == 2 * distA[b]), "Distance check failed"

    # Calculate paths between A, B, and C
    AB = np.sqrt((losses[b] - losses[a]) ** 2 + distA[b] ** 2)
    BC = np.sqrt((losses[c] - losses[b]) ** 2 + distA[b] ** 2)
    AC = np.sqrt((losses[c] - losses[a]) ** 2 + distA[c] ** 2)

    # Roghness is how much the path via B (A->B->C) is longer than the direct path A->C
    # subtracting 1, then we can use log scale better!
    roughness = (AB + BC) / AC - 1

    # Update DataFrame with roughness values
    df.loc[(direction,), 'Roughness'] = np.concatenate([[np.nan], roughness, [np.nan]])

df

Model  \
Direction    Distance                                                      
Ascent       0.000000                                                NaN   
             0.062500  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.125000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.250000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.500000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.000000  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Descent      0.000000                                                NaN   
             0.074325  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.148651  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.297302  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.594604  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.189207  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Radially In  0.000000                                                NaN   
             0.105112  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.210224  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.420448  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.840896  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.681793  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
Radially Out 0.000000                                                NaN   
             0.088388  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.176777  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.353553  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             0.707107  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   
             1.414214  ResNet(\n  (conv1): Conv2d(3, 64, kernel_size=...   

                            Loss     Roughness  
Direction    Distance                           
Ascent       0.000000   2.413582           NaN  
             0.062500   2.659799  4.423994e-04  
             0.125000   2.987322  5.610045e-04  
             0.250000   3.835409  4.132437e-04  
             0.500000   5.992914  1.538731e-04  
             1.000000  10.814508           NaN  
Descent      0.000000   2.413582           NaN  
             0.074325   2.250852  1.183501e-01  
             0.148651   2.242186  7.502522e-01  
             0.297302   2.588072  5.522263e-02  
             0.594604   3.985614  3.944146e-03  
             1.189207   7.398079           NaN  
Radially In  0.000000   2.413582           NaN  
             0.105112   2.412963  6.173595e-11  
             0.210224   2.412346  2.493590e-10  
             0.420448   2.411118  9.836014e-10  
             0.840896   2.408692  3.881616e-09  
             1.681793   2.403950           NaN  
Radially Out 0.000000   2.413582           NaN  
             0.088388   2.414105  4.629208e-11  
             0.176777   2.414629  1.753595e-10  
             0.353553   2.415683  7.156116e-10  
             0.707107   2.417811  2.899024e-09  
             1.414214   2.422147           NaN

In [37]:
# new nomenclature: grit := scale dependent roughness

# Create a Plotly figure
fig = go.Figure()

# Plotting roughness for each direction using grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Roughness'],
        mode='markers',
        name=direction
    ))

# Set axis labels, title, and configure log scale on x-axis
fig.update_layout(
    title=f'Scale Dependent Roughness of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis={
        'title': 'Coarse Graining Scale',
        'type': 'log',
        'dtick': 1,  # Tick every power of ten
    },
    yaxis ={
        'title': 'Roughness',
        'type': 'log',
    },
    legend_title='Direction',
    template='seaborn',
    height=600
)

# Display the figure
fig.show()
